<h1> xlsx -> csv 변환

In [ ]:
import pandas as pd

def save_sheets_as_csv(excel_file_path):
    # 엑셀 파일 읽기
    excel_file = pd.ExcelFile(excel_file_path)
    
    # 각 시트 이름을 가져오기
    sheet_names = excel_file.sheet_names
    
    # 각 시트를 CSV 파일로 저장
    for sheet_name in sheet_names:
        # 시트 읽기
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
        
        # CSV 파일 이름 생성 (시트 이름 + .csv)
        csv_file_path = f"{sheet_name}.csv"
        
        # 데이터프레임을 CSV 파일로 저장
        df.to_csv(csv_file_path, index=False)
        print(f"Saved {sheet_name} to {csv_file_path}")

# 사용 예시
excel_file_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/gt_v1.xlsx'
save_sheets_as_csv(excel_file_path)


In [ ]:
import pandas as pd
import numpy as np

def process_csv(file_path, output_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, parse_dates=['datetime'])

    # datetime을 인덱스로 설정
    df.set_index('datetime', inplace=True)

    # 5분 간격으로 데이터 묶기
    grouped = df.resample('5T')

    # 숫자형 데이터의 통계 계산 함수
    # 최소, 최대, 평균, 중앙, Q1, Q3, IQR
    def calculate_statistics(group):
        stats = {
            'min': group.min(),
            'max': group.max(),
            'mean': group.mean(),
            'median': group.median(),
            'Q1': group.quantile(0.25),
            'Q3': group.quantile(0.75),
            'IQR': group.quantile(0.75) - group.quantile(0.25)
        }
        return stats

    # 결과를 저장할 데이터프레임 생성
    result = []

    for time, group in grouped:
        # 숫자형 데이터 통계 계산
        stats = calculate_statistics(group.select_dtypes(include=[np.number]))
        tuple_stats = {col: (stats['min'][col], stats['max'][col], stats['mean'][col],
                             stats['median'][col], stats['Q1'][col], stats['Q3'][col],
                             stats['IQR'][col]) for col in stats['min'].index}
        
        # 튜플을 리스트로 변환하여 데이터프레임 생성
        tuple_stats_df = pd.DataFrame([tuple_stats], index=[time])

        # 객체 타입 데이터 처리 (변경 여부 확인)
        obj_changes = group.select_dtypes(include=[object]).apply(lambda x: 1 if x.nunique() > 1 else 0)
        obj_changes_df = pd.DataFrame([obj_changes], index=[time])

        # 숫자형 통계와 객체 타입 변경 여부 결합
        combined_df = tuple_stats_df.join(obj_changes_df)

        result.append(combined_df)

    # 결과 데이터프레임 생성
    result_df = pd.concat(result)

    # 인덱스를 열로 변환하고 이름을 datetime으로 설정
    result_df.reset_index(inplace=True)
    result_df.rename(columns={'index': 'datetime'}, inplace=True)

    # 결과 CSV 파일로 저장
    result_df.to_csv(output_path, index=False)


# 사용 예시
file_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/3월24일.csv'
output_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/테스트용.csv'
process_csv(file_path, output_path)


In [ ]:
# 데이터 타입 확인
file_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/3월24일.csv'
df = pd.read_csv(file_path)

print(df.dtypes)

In [ ]:
import pandas as pd

def csv_to_dataframe(file_path):
    # CSV 파일을 읽어 DataFrame으로 변환 (datetime 열을 날짜 형식으로 파싱하고 인덱스로 설정)
    df = pd.read_csv(file_path, parse_dates=['datetime'], index_col='datetime')
    return df

# 사용 예시
file_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/테스트용.csv'
df = csv_to_dataframe(file_path)

# DataFrame 출력
print(df.head(1))

In [ ]:
import pandas as pd

def view_specific_columns(file_path, column_names):
    # CSV 파일을 읽어 DataFrame으로 변환
    df = pd.read_csv(file_path)
    
    # 특정 컬럼들의 값 확인
    columns_values = df[column_names]
    return columns_values

# 사용 예시
file_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/테스트용.csv'
column_names = ['G1 START', 'G1 STOP', 'G1 GT TRIP']
columns_values = view_specific_columns(file_path, column_names)

# 특정 컬럼들의 값 출력
print(columns_values)


print(df.dtypes)

In [13]:
import pandas as pd
import sqlite3

def csv_to_sqlite(file_path, db_path, table_name='test_table'):
    # CSV 파일을 읽어 DataFrame으로 변환
    df = pd.read_csv(file_path)

    # SQLite 데이터베이스 연결
    conn = sqlite3.connect(db_path)
    
    # DataFrame을 SQLite 테이블로 저장
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    # 데이터베이스 연결 종료
    conn.close()

# 사용 예시
file_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/테스트용.csv'
db_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/test_database.db'
csv_to_sqlite(file_path, db_path)

print("CSV 파일이 SQLite 데이터베이스로 성공적으로 변환되었습니다.")

CSV 파일이 SQLite 데이터베이스로 성공적으로 변환되었습니다.


In [ ]:
import sqlite3

def get_table_schema(db_path, table_name):
    # SQLite 데이터베이스 연결
    conn = sqlite3.connect(db_path)
    
    # 커서 객체 생성
    cursor = conn.cursor()
    
    # 테이블 스키마 조회 쿼리 실행
    cursor.execute(f"PRAGMA table_info({table_name});")
    
    # 결과 가져오기
    schema = cursor.fetchall()
    
    # 데이터베이스 연결 종료
    conn.close()
    
    # 스키마 출력
    for column in schema:
        print(column)

# 사용 예시
db_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/test_database.db'
table_name = 'test_table'
get_table_schema(db_path, table_name)

In [8]:
import pandas as pd

# CSV 파일 경로
file_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/3월24일.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 데이터프레임의 상위 10개 행 추출
df = df.head(10)

# datetime 컬럼을 제외한 object 타입의 열 찾기
object_columns = df.select_dtypes(include=['object']).columns.tolist()
object_columns.remove('datetime')

# object 타입 열과 해당 열이 포함된 행 삭제
df = df.drop(columns=object_columns).dropna()

# 수정된 데이터 저장
output_file_path = '/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/modified_3월24일.csv'
df.to_csv(output_file_path, index=False)

print(f"Modified CSV saved to {output_file_path}")

Modified CSV saved to /workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/modified_3월24일.csv
